In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [62]:
gscores=pd.read_csv('genome_scores.csv')
ratings=pd.read_csv('rating.csv')
movies=pd.read_csv('movie.csv')
#tags=pd.read_csv('tag.csv')
#link=pd.read_csv('link.csv')
gtags=pd.read_csv('genome_tags.csv')

In [63]:
gscores=gscores[gscores['relevance']>0.6]

In [64]:
gscores=gscores.merge(gtags,on='tagId') 

In [65]:
gscores=gscores.merge(movies,on='movieId')

In [66]:
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [67]:
gscores=splitDataFrameList(gscores,"genres","|")

In [68]:
gscores.drop(columns=['movieId','tagId'],inplace=True)

In [69]:
#gscores['rating']=ratings['rating']

In [70]:
movie_pivot=gscores.pivot_table(index='title',columns='tag',aggfunc='mean')

In [71]:
movie_pivot.fillna(-1,inplace=True)

In [72]:
movie_pivot

relevance                            \
tag                                             007 007 (series) 18th century   
title                                                                           
$9.99 (2008)                               -1.00000     -1.00000         -1.0   
'71 (2014)                                 -1.00000     -1.00000         -1.0   
'Round Midnight (1986)                     -1.00000     -1.00000         -1.0   
'Til There Was You (1997)                  -1.00000     -1.00000         -1.0   
'burbs, The (1989)                         -1.00000     -1.00000         -1.0   
...                                             ...          ...          ...   
eXistenZ (1999)                            -1.00000     -1.00000         -1.0   
xXx (2002)                                  0.90675      0.95225         -1.0   
xXx: State of the Union (2005)             -1.00000     -1.00000         -1.0   
¡Three Amigos! (1986)                      -1.00000     -1.00000         -1.0   
À nous la liberté (Freedom for Us) (1931)  -1.00000     -1.00000         -1.0   

                                                                            \
tag                                       1920s    1930s 1950s 1960s 1970s   
title                                                                        
$9.99 (2008)                               -1.0 -1.00000  -1.0  -1.0  -1.0   
'71 (2014)                                 -1.0 -1.00000  -1.0  -1.0  -1.0   
'Round Midnight (1986)                     -1.0 -1.00000  -1.0  -1.0  -1.0   
'Til There Was You (1997)                  -1.0 -1.00000  -1.0  -1.0  -1.0   
'burbs, The (1989)                         -1.0 -1.00000  -1.0  -1.0  -1.0   
...                                         ...      ...   ...   ...   ...   
eXistenZ (1999)                            -1.0 -1.00000  -1.0  -1.0  -1.0   
xXx (2002)                                 -1.0 -1.00000  -1.0  -1.0  -1.0   
xXx: State of the Union (2005)             -1.0 -1.00000  -1.0  -1.0  -1.0   
¡Three Amigos! (1986)                      -1.0 -1.00000  -1.0  -1.0  -1.0   
À nous la liberté (Freedom for Us) (1931)  -1.0  0.81175  -1.0  -1.0  -1.0   

                                                              ...  \
tag                                       1980s 19th century  ...   
title                                                         ...   
$9.99 (2008)                               -1.0       -1.000  ...   
'71 (2014)                                 -1.0       -1.000  ...   
'Round Midnight (1986)                     -1.0       -1.000  ...   
'Til There Was You (1997)                  -1.0       -1.000  ...   
'burbs, The (1989)                         -1.0       -1.000  ...   
...                                         ...          ...  ...   
eXistenZ (1999)                            -1.0       -1.000  ...   
xXx (2002)                                 -1.0       -1.000  ...   
xXx: State of the Union (2005)             -1.0       -1.000  ...   
¡Three Amigos! (1986)                      -1.0       -1.000  ...   
À nous la liberté (Freedom for Us) (1931)  -1.0        0.602  ...   

                                                                      \
tag                                       world politics world war i   
title                                                                  
$9.99 (2008)                                        -1.0        -1.0   
'71 (2014)                                          -1.0        -1.0   
'Round Midnight (1986)                              -1.0        -1.0   
'Til There Was You (1997)                           -1.0        -1.0   
'burbs, The (1989)                                  -1.0        -1.0   
...                                                  ...         ...   
eXistenZ (1999)                                     -1.0        -1.0   
xXx (2002)                                          -1.0        -1.0   
xXx: State of the Union (2005)                      -1.

In [73]:
from scipy.sparse import csr_matrix
movie_matrix=csr_matrix(movie_pivot)

In [74]:
movie_matrix

<10379x1126 sparse matrix of type '<class 'numpy.float64'>'
	with 11686754 stored elements in Compressed Sparse Row format>

In [75]:
from sklearn.neighbors import NearestNeighbors
NN_tags=NearestNeighbors(algorithm='brute')

In [76]:
NN_tags.fit(movie_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [77]:
def movie_recommender(movie):
    distance,suggestions=NN_tags.kneighbors(movie_pivot[movie_pivot.index==movie].values.reshape(1,-1),n_neighbors=10)
    for i in suggestions[0]:
        print(movie_pivot.index[i])

# Example 1 

In [78]:
movie_recommender('Batman Begins (2005)')

Batman Begins (2005)
Dark Knight Rises, The (2012)
Dark Knight, The (2008)
Batman: The Dark Knight Returns, Part 1 (2012)
Batman: The Dark Knight Returns, Part 2 (2013)
Batman: Under the Red Hood (2010)
Superman/Batman: Apocalypse (2010)
Batman (1989)
Batman Beyond: Return of the Joker (2000)
Batman: Mask of the Phantasm (1993)


# Example 2

In [79]:
movie_recommender("Harry Potter and the Chamber of Secrets (2002)")

Harry Potter and the Chamber of Secrets (2002)
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
Harry Potter and the Prisoner of Azkaban (2004)
Harry Potter and the Goblet of Fire (2005)
Harry Potter and the Order of the Phoenix (2007)
Harry Potter and the Half-Blood Prince (2009)
Harry Potter and the Deathly Hallows: Part 1 (2010)
Chronicles of Narnia: Prince Caspian, The (2008)
Spiderwick Chronicles, The (2008)
Seeker: The Dark Is Rising, The (2007)


# Example 3

In [86]:
movie_recommender("Song of the Little Road (Pather Panchali) (1955)")

Song of the Little Road (Pather Panchali) (1955)
Unvanquished, The (Aparajito) (1957)
World of Apu, The (Apur Sansar) (1959)
Mouchette (1967)
Promise, The (La promesse) (1996)
Canterbury Tale, A (1944)
Mirror, The (Zerkalo) (1975)
Way Home, The (Jibeuro) (2002)
I Can't Sleep (J'ai pas sommeil) (1994)
Time Out (L'emploi du temps) (2001)
